__Before you start working on this notebook, save a copy to Google Drive!__

<h5 class='prehead'>SM286D &middot; Introduction to Applied Mathematics with Python &middot; Spring 2020 &middot; Uhan</h5>

<h5 class='lesson'>Lesson 13.</h5>

<h1 class='lesson_title'>Social network analysis</h1>

## This lesson...

We're going to find the key conspirators in the American Revolution through social network analysis. In particular, we will reproduce parts this analysis by Prof. Kieran Healy:

https://kieranhealy.org/blog/archives/2013/06/09/using-metadata-to-find-paul-revere/

(If the link above doesn't work, you can look at this [PDF version](https://github.com/sm286d-uhan/social-network-analysis/raw/colab/FindingPaulRevere.pdf) instead.)

Prof. Healy used R to perform his analysis. Naturally, we will use Python.

---

### Part 0

First, read Prof. Healy's analysis linked above to get an idea of what's coming next. It's a fun read!

### Part 1

Ordinarily, we would use Excel and xlwings for this part. Let's see how we can accomplish the same thing using Google apps instead.

Take a look at [this Google Sheet workbook](https://docs.google.com/spreadsheets/d/1EKLwS-E-QZnuCfg1kHucx4jHqLEdehP9VOPAFCZQShc/edit?usp=sharing).

You'll see that the first sheet `PaulRevereAppD` has a table with oddly formatted names of revolutionaries in column A. In the next 7 columns we have a 0-1 matrix indicating whether the revolutionary belonged to one of seven clubs and social groups (1 if they did belong and 0 if they didn't). 

#### Accessing a Google Spreadsheet from a Google Colab notebook

The code below handles authentication and imports the `gspread` library which lets you interact with Google Sheets using Python. After running the cell you'll be prompted to enter in some credentials, which Google will give you by following the link provided. 

_Note._ For more details, take a look at [the section on Google Sheets from this very helpful Colab walkthrough on accessing external data](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=sOm9PFrT8mGG).

In [0]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials

# Authenticate Google user
auth.authenticate_user()

# Create Google Sheets object
gc = gspread.authorize(GoogleCredentials.get_application_default())

To interact with a Google Sheet workbook, all you need is the URL: use the `.open_by_url()` method of `gc`. (Recall that  `gc` is the Google Sheets object we defined above.) 

In [0]:
# Create workbook object from Google Sheets workbook URL
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1EKLwS-E-QZnuCfg1kHucx4jHqLEdehP9VOPAFCZQShc/edit?usp=sharing')

Now that you have the workbook object, you can access specific worksheets. Note that the workbook you're working with has only one sheet which makes this really easy. However, if your workbook had, for instance, 10 different sheets or tabs, then it would be nice to know how to call one specific sheet. 

You can use the `.worksheet()` method of the workbook object `wb`, like this:

In [0]:
# Create worksheet object from worksheet that contains the data
sheet = wb.worksheet('PaulRevereAppD')

To extract all of the values from this worksheet sheet, you can call the `.get_all_values()` method on the worksheet object `sheet`, like this:

In [0]:
# Get all the data from the sheet
data = sheet.get_all_values()

What do you get? Print `data` to check your work.

In [5]:
# Check your work
print(data)

[['Adams.John', '0', '0', '1', '1', '0', '0', '0'], ['Adams.Samuel', '0', '0', '1', '1', '0', '1', '1'], ['Allen.Dr', '0', '0', '1', '0', '0', '0', '0'], ['Appleton.Nathaniel', '0', '0', '1', '0', '0', '1', '0'], ['Ash.Gilbert', '1', '0', '0', '0', '0', '0', '0'], ['Austin.Benjamin', '0', '0', '0', '0', '0', '0', '1'], ['Austin.Samuel', '0', '0', '0', '0', '0', '0', '1'], ['Avery.John', '0', '1', '0', '0', '0', '0', '1'], ['Baldwin.Cyrus', '0', '0', '0', '0', '0', '0', '1'], ['Ballard.John', '0', '0', '1', '0', '0', '0', '0'], ['Barber.Nathaniel', '0', '0', '1', '0', '1', '1', '1'], ['Barnard.Samuel', '0', '0', '0', '0', '1', '0', '0'], ['Barrett.Samuel', '1', '0', '0', '0', '0', '0', '1'], ['Bass.Henry', '0', '1', '1', '0', '1', '0', '1'], ['Bell.William', '1', '0', '0', '0', '0', '0', '0'], ['Blake.Increase', '1', '0', '0', '0', '0', '0', '0'], ['Boit.John', '0', '0', '1', '0', '0', '0', '0'], ['Bolter.Thomas', '0', '0', '0', '0', '1', '0', '0'], ['Boyer.Peter', '0', '0', '0', '0', '

You should see that `data` is a list of lists. Each inner list represents a row of the worksheet. To get a single list that contains the names of the revolutionaries, you can use a list comprehension, like this:

In [0]:
# Use a list comprehension to get the names of the revolutionaries in a single list
names = [data[i][0] for i in range(len(data))]

Check your work: who are the first two revolutionaries?

In [7]:
# Check your work
print(f"The two revolutionary cousins are {names[0]} and {names[1]}.")

The two revolutionary cousins are Adams.John and Adams.Samuel.


### Part 2

Now get the 0-1 matrix from the sheet `PaulRevereAppD`. 

Start by creating a list of lists that only stores the floating point versions of the numeric values from the spreadsheet. Use `data` from Part 1 to do this.

In [8]:
# Grab 0-1 matrix from the variable data we created above
zero_one_values = []
for i in range(len(data)):
    row = []
    
    for j in range(1, len(data[i])):
        row.append(float(data[i][j]))
        
    zero_one_values.append(row)
    
# Check our work
print(zero_one_values)

[[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 

Next, store the list of lists you created in a NumPy array `A`. Use the `.shape` attribute to find the dimensions of the array `A`. 

Check your work: you should have a NumPy array with 254 rows and 7 columns.

In [9]:
import numpy as np

# Put 0-1 matrix into a Numpy array
A = np.array(zero_one_values)

# Print dimensions of the matrix
print(f"The dimensions of A are {A.shape}.")

The dimensions of A are (254, 7).


### Part 3

Now we're going to create the adjacency matrix:

- Use the `@` operator and the `.transpose()` method to create the adjacency matrix $M = AA^T$.
- Print the matrix and take a look. It contains some entries that are different from 0 and 1. 
- Replace nonzero entries with 1s. Also, replace all diagonal entries with 0s since each node should not be connected to itself. Check your work by printing the matrix again.

_Pro tip._ A slick way to get the number of rows and columns of a NumPy array is:

```python
# m = number of rows
# n = number of columns
m, n = A.shape
```

Alternatively, you can use `A.shape[0]` and `A.shape[1]`.

In [10]:
# Compute M
M = A @ A.transpose()

# Check our work
print(M)

[[2. 2. 1. ... 1. 0. 1.]
 [2. 4. 1. ... 2. 0. 2.]
 [1. 1. 1. ... 1. 0. 1.]
 ...
 [1. 2. 1. ... 2. 0. 1.]
 [0. 0. 0. ... 0. 1. 1.]
 [1. 2. 1. ... 1. 1. 3.]]


In [11]:
# Get size of matrix
m, n = M.shape

# Modify adjacency matrix
for i in range(m):
    for j in range(n):
        # Replace nonzero entries with 1s
        if M[i, j] > 0:
            M[i, j] = 1
        
        # Replace diagonal entries with 0s
        if i == j:
            M[i, j] = 0

# Check our work
print(M)

[[0. 1. 1. ... 1. 0. 1.]
 [1. 0. 1. ... 1. 0. 1.]
 [1. 1. 0. ... 1. 0. 1.]
 ...
 [1. 1. 1. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]]


### Part 4

Use NetworkX to make a graph `G` whose adjacency matrix is `M`. Recall that `G.degree()[i]` gives you the degree of node `i`. Make a list `degs` whose $i^\text{th}$ entry is the degree of node $i$. Check your work: `degs[101]` should equal 135. 

In [12]:
import networkx as nx

# Create graph
G = nx.Graph(M)

# Create list of degrees
# Recall that n is the number of rows of M, 
# which is the number of nodes in G
# Here, we've used a list comprehension
# Using a for loop with .append() is OK too
degs = [G.degree()[i] for i in range(n)]

# Check our work
print(degs[101])

135


### Part 5

Define $N$ to be the number of revolutionaries, or vertices in `G`. The _normalized degree_ (also known as *degree centrality*) of vertex $i$ is the degree of vertex $i$ divided by $N-1$. Using the list `degs` you created in Part 4, make a list `ndegs` whose $i^\text{th}$ entry is the normalized degree of node $i$. Check your work: `ndegs[101]` should be approximately 0.5336.

(Recall from Lesson 12 that we could have used `nx.degree_centrality()` to compute these values. Let's do this "manually" here instead.)

In [13]:
# Compute N
N = len(G.nodes)

# Compute normalized degree
ndegs = [d / (N - 1) for d in degs]

# Check our work
print(ndegs[101])

0.5335968379446641


### Part 6

We want to print out a list of the five revolutionaries with the highest normalized degrees. We'll do this over the next few parts.

First, let's see how to print a nicely formatted string. Run the following cell:

In [14]:
# Define variables for  name, rank and value
nm = 'Will'
rk = 2
val = 3.1415

# Print this information
print(f'{nm:<5s} {rk:>5d} {val:0.3f}')

Will      2 3.142


Note that the f-string looks different from the ones we've used so far &mdash; in particular, what's inside the `{ }`.

The expressions inside `{ }` before `:` are evaluated at run time, and in this case will take on the values of `Will`, `2`, and `3.1415`, respectively.  Python prints the value for each of these variables. 

The `5s` following the first `:` means that the value should be formatted as a string (that's due to the `s`) and that the string should take at least 5 characters. The `<` sign tells Python to left justify the string in the 5 character space. If the string is longer than 5 characters, all characters are printed; the 5 just determines the padding that we'll use if the string is shorter than 5 characters. This is very helpful in lining up printed text in nice columns. 

The value for `rk` is formatted as an integer (that's due to the `d`, think: digit) with at least 5 spaces. The `>` sign tells Python to right justify the string in the 5 character space. If you want your string to be centered in the allotted character space, you should use `^` in place of the `<` or `>`.  

Finally, `val` is formatted as a float (that's due to the `f`). The minimum field is 0 characters but the value following the decimal is important too: it is the precision, the number of characters after the decimal that are printed. If the number has more decimals, the value is rounded. 
   
[Here's more reading about f-strings.](https://realpython.com/python-f-strings/)

Use code similar to the code given above to print the first American revolutionary's name (left justified, in at least 25 spaces), their rank (1), and their normalized degree (to 4 decimal places accuracy). It might make more sense to print their rank first. Also, print a header with the words `Rank`, `Name`, and `ndeg` on top by using a second print statement. Your output should look like:

```
Rank  Name                      ndeg
1     Adams.John                0.2885
```

In [15]:
# Print table heading
rank = 'Rank'
name = 'Name'
measure = 'ndeg'
print(f'{rank:5s} {name:<25s} {measure:s}')

# Print information for first revolutionary
rank = 1
name = names[0]
measure = ndegs[0]
# Note: < is for left aligned, > is for right aligned, ^ is for center aligned
print(f'{rank:<5d} {name:<25s} {measure:0.4f}') 

Rank  Name                      ndeg
1     Adams.John                0.2885


### Part 7

Now we'd like to sort the list of normalized degrees. We could just use the `sorted` function to do this, but it would help if we knew how the original list was permuted or rearranged. The `np.argsort` method does just that! 

Take a look at this example code:

In [16]:
# Define sample list of values
list_of_vals = [1, 3, 2]

# Get indices of how the values should be sorted
idx = np.argsort(list_of_vals)[::-1]

# What do the indices look like?
print(idx)

# Check our work with sorted()
print([list_of_vals[i] for i in idx] == sorted(list_of_vals, reverse=True))

[1 2 0]
True


On line 5, we pass `list_of_vals` to the `np.argsort` method, which returns a list-like object (a NumPy array, to be exact). 

Recall the slicing syntax: `[start:stop:step]`. So adding `[::-1]` to a list-like object reverses the order of the list. Since `np.argsort` sorts by increasing order, we want to reverse the order of the list it returns.

Note that

```python
list_of_vals[1] > list_of_vals[2] > list_of_vals[0]
```

so the output of `np.argsort` is a list of the indices for the original list that produce the sorted list. We verify this on line 11, which outputs `True`. 

Now you should find the indices that sort the normalized degrees from highest to lowest. Check your work: your first 3 values should be 199, 10 and 110. Also find the names of the American revolutionaries with the highest 3 normalized degrees. 

In [17]:
# Get indices of how the normalized degree 
# values should sorted, highest to lowest
idx = np.argsort(ndegs)[::-1]

# Print the indices corresponding to the 
# 3 highest normalized degree values
print(idx[0:3])

# Print the names of the revolutionaries with the
# 3 highest normalized degree values
print('The American revolutionaries with the three highest ndegs values are:')
print(names[idx[0]])
print(names[idx[1]])
print(names[idx[2]])

[199  10 110]
The American revolutionaries with the three highest ndegs values are:
Revere.Paul
Barber.Nathaniel
Hoffins.John


### Part 8

Finally, put the last two parts together to produce a chart with the top 5 American revolutionaries, ranked by normalized degree. Your chart should have columns for rank, name and normalized degree. It should have a row for each of the five revolutionaries.

_Challenge._ Can you write the names so that they don't have the odd formatting? Hint: use the `.split()` method for strings, described [here in the documentation](https://docs.python.org/3.7/library/stdtypes.html#str.split).

In [18]:
# Print table heading
rank = 'Rank'
name = 'Name'
measure = 'ndeg'
print(f'{rank:5s} {name:25s} {measure:s}')

# Print top 5 revolutionaries by normalized degree
for i in range(5):
    name_split = names[idx[i]].split('.')
    full_name = f'{name_split[1]} {name_split[0]}'
    print(f'{(i+1):<5d} {full_name:25s} {ndegs[idx[i]]:0.4f}')

Rank  Name                      ndeg
1     Paul Revere               0.9802
2     Nathaniel Barber          0.7905
3     John Hoffins              0.7905
4     William Cooper            0.7905
5     William Greenleaf         0.7826


### Part 9

Now we're going to package all our work from the last three parts into a function. 

Define a function called `foutput` that takes inputs `measure_list`, `measure_name`, `name_list` and `many`:

- `measure_list` should be a list of measurements, not necessarily sorted. 
- `measure_name` should be a string with the name of the quantity measured in `measure_list`.
- `name_list` should be a list of names, one for each number in `measure_list`. 
- `many` should be a number and it should be equal to 5 as a default value. 

The function should print a table. The table should have columns for rank, name and the measured quantity. You should print the top `many` values of `measure_list`, one on each row, with the corresponding names. 

Test your function on your normalized degree data to ensure that it gives the same output as in the previous question. Also test your function if you don't give a value for `many` as input.

In [19]:
def foutput(measure_list, measure_name, name_list, many=5):
    """
    Print top `many` values of `measurelist`, one on each row, with the corresponding names. 
    """
    # Get indices of how the measure list should sorted, highest to lowest
    idx = np.array(measure_list).argsort()[::-1]
    
    # Print table heading
    rank = 'Rank'
    name = 'Name'
    print(f'{rank:5s} {name:25s} {measure_name:s}')
    
    # Print top `many` names by measure
    for i in range(many):
        name_split = name_list[idx[i]].split('.')
        full_name = f'{name_split[1]} {name_split[0]}'
        print(f'{(i+1):<5d} {full_name:25s} {measure_list[idx[i]]:0.4f}')
        
# Test function        
foutput(ndegs, 'ndeg', names, 5) 

Rank  Name                      ndeg
1     Paul Revere               0.9802
2     Nathaniel Barber          0.7905
3     John Hoffins              0.7905
4     William Cooper            0.7905
5     William Greenleaf         0.7826


### Part 10

The *normalized closeness* (also known as *closeness centrality*) is another measure of the centrality of nodes in a network (see Lesson 12 and Project 5 for details). Recall that NetworkX has a built-in function to compute normalized closeness: 

In [0]:
# Compute closeness centrality
# nx.closeness_centrality returns a dictionary
closeness = nx.closeness_centrality(G)

# Create list of closeness centrality values
# Recall that n is the number of nodes in G
# Here, we've used a list comprehension
# Using a for loop with .append() is OK too
closeness_list = [closeness[i] for i in range(n)]

The code above produces a list `closeness_list` that contains the normalized closeness values. Use your code from the last part to produce a table with the names and scores of the top five American revolutionaries as measured by normalized closeness. 

In [21]:
foutput(closeness_list, 'closeness', names, 5)

Rank  Name                      closeness
1     Paul Revere               0.9806
2     Nathaniel Barber          0.8268
3     John Hoffins              0.8268
4     William Cooper            0.8268
5     William Greenleaf         0.8214


### Part 11

Computing the normalized closeness requires us to find all the shortest paths between two people in our relationship graph. 

The NetworkX method `nx.shortest_path` computes a shortest path between two nodes in a graph. Look at the NetworkX documentation to find out how to use this method. Then find a shortest path from Nathaniel Barber (node 10 in our graph) to John Winslow (node 250 in our graph). 

Next, use the NetworkX method `nx.all_shortest_paths` to count the number of shortest paths linking Nathaniel Barber to John Winslow. 

In [22]:
# Find a shortest path betwteen nodes 10 and 250
# Print the path (node numbers)
path = nx.shortest_path(G, source=10, target=250)
print(f'A shortest path between nodes 10 and 250 is: {path}')

# Print the corresponding names of this path
npath = [names[i] for i in path]
print(f'A shortest path between Nathaniel Barber and John Winslow is: {npath}')

# Find all shortest paths between nodes 10 and 250
# Print the paths
# Count the paths
paths = list(nx.all_shortest_paths(G, source=10, target=250))
print('All of the shortest paths between Nathaniel Barber and John Winslow:')
print(paths)
print(f'There are actually {len(paths)} shortest paths between Nathaniel Barber and John Winslow.')

A shortest path between nodes 10 and 250 is: [10, 0, 250]
A shortest path between Nathaniel Barber and John Winslow is: ['Barber.Nathaniel', 'Adams.John', 'Winslow.John']
All of the shortest paths between Nathaniel Barber and John Winslow:
[[10, 0, 250], [10, 1, 250], [10, 50, 250], [10, 51, 250], [10, 100, 250], [10, 101, 250], [10, 166, 250], [10, 197, 250], [10, 199, 250], [10, 235, 250]]
There are actually 10 shortest paths between Nathaniel Barber and John Winslow.


### Part 12

The PageRank algorithm is one of the celebrated algorithms of the internet age. It ranks the importance of web pages on the internet. It can also be used to rank the importance of nodes in a network. The algorithm sets up a transition matrix for some evolving process on the graph and then computes the eigenvector corresponding to the maximum eigenvalue of this matrix. More details are in Project 5.

The code below computes the PageRank of each node in our network of American revolutionaries. 

In [0]:
pr = nx.pagerank(G, alpha=0.85)

`nx.pagerank()` has a parameter `alpha` that is customarily set to 0.85 but other values are also possible. Print a table with the names and scores of the top five American revolutionaries as measured by PageRank. 

In [24]:
# Compute PageRank
# nx.pagerank returns a dictionary
pr = nx.pagerank(G, alpha=0.85)

# Create list of PageRank values
# Recall that n is the number of nodes in G
# Here, we've used a list comprehension
# Using a for loop with .append() is OK too
pr_list = [pr[i] for i in range(n)]

# Print top 5 revolutionaries by PageRank
foutput(pr_list, 'PageRank', names, 5)

Rank  Name                      PageRank
1     Paul Revere               0.0113
2     Joseph Warren             0.0087
3     John Hoffins              0.0087
4     William Cooper            0.0087
5     Nathaniel Barber          0.0087


### Part 13

**This question is considered challenge material, BUT it is important because you need to do something similar on your assignment.** 

Let's delve into the PageRank method a bit further. In the code cell below, we define the matrix `MM`, which follows the formula given for $M$ in the PageRank section of Project 5.

In [0]:
# Number of nodes
N = G.number_of_nodes()

# Set damping parameter
d = 0.85

# Form matrix K whose diagonal entries = out degree
# We can accomplish this by summing the entries in each row or column of A
out_degree = [sum(M[:, i]) for i in range(N)]
K = np.diag(out_degree)

# Compute PageRank matrix
MM = d * (np.linalg.inv(K) @ M).transpose() + ((1 - d) / N) * np.ones([N, N])

The method we use to find the largest eigenvalue is an _iterative_ method that starts with a _random_ vector with components between 0 and 1. We divide the vector by its 1-norm (the sum of the absolute values of its components) so that it represents a probability distribution on the set of nodes. 

In [0]:
# Make random vector v with components between 0 and 1
v = np.random.rand(N)

# Divide the vector v by its 1-norm
v = v / np.linalg.norm(v, ord=1)

Now we set a small error threshold (`epsilon`) and initialize our error to a large overestimate (`error`). We iterate until the error between our current vector `v` and the previous version of vector `v` is smaller than the error threshold. At each iteration we multiply `v` by `MM`, and rescale `v` by dividing by its 1-norm (to keep it a probability distribution). We compute the error using the 2-norm.

In [0]:
# Iteratively improve v     
epsilon = 1e-6
error = float("inf")
iteration = 0

while error > epsilon:  # stop when error <= epsilon
    iteration += 1
    oldv = v
    v = MM @ v
    v = v / np.linalg.norm(v, ord=1)
    error = np.linalg.norm(v - oldv, ord=2)

First, create a NumPy array `R` consisting of the values computed by `nx.pagerank()`, so that `R[i]` is the PageRank of revolutionary `i`. (You should have already created a list of these values in Part 12. How do you convert the list into a one-dimensional NumPy array?)

Then, compute the 2-norm distance between the array `v` output by this process and values computed by `nx.pagerank()`.

Check your work: the distance should be close to $5 \times 10^{-5}$. In general the vector $v$ is close to the true eigenvector $R$ though it may not be quite as close as the error tolerance.  

In [28]:
# Compare our v with the output from pagerank
R = np.array(pr_list)
diff = np.linalg.norm(v - R, ord=2)
print(f"The difference in 2-norm between our v and the output from the Networkx PageRank algorithm is {diff}.")

The difference in 2-norm between our v and the output from the Networkx PageRank algorithm is 5.562490403141825e-05.
